#
#### Author: Pavel Khudov


In [186]:
import pandas as pd
import numpy as np
import re

In [187]:
# Read data
dirty_listings_df = pd.read_csv('https://www.inf.ed.ac.uk/teaching/courses/fds/data/project-2022-2023/airbnb/listings.csv.gz')
dirty_listings_df.head()

,id,listing_url,scrape_id,last_scraped,source,name,description,neighborhood_overview,picture_url,host_id,...,review_scores_communication,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,15420,https://www.airbnb.com/rooms/15420,20221216161317,2022-12-16,city scrape,Georgian Boutique Apt City Centre,"Stunning, spacious ground floor apartment minu...","The neighbourhood is in the historic New Town,...",https://a0.muscache.com/pictures/cf69631f-4194...,60423,...,4.98,4.98,4.91,NaN,f,1,1,0,0,3.10
1,707097,https://www.airbnb.com/rooms/707097,20221216161317,2022-12-16,city scrape,Centre Royal Mile Apartment 3 bedrooms 2 bathr...,NaN,"The location is the perfect for tourism , shor...",https://a0.muscache.com/pictures/6e2ded8d-f20b...,3092851,...,4.72,4.94,4.09,NaN,t,8,6,2,0,0.55
2,728199,https://www.airbnb.com/rooms/728199,20221216161317,2022-12-16,city scrape,"Private room in central, spacious and comfy flat",Fantastic main door flat over two levels withi...,"Great location for access to the city centre, ...",https://a0.muscache.com/pictures/11315577/0091...,3776412,...,4.97,4.73,4.79,NaN,f,1,0,1,0,2.86
3,732008,https://www.airbnb.com/rooms/732008,20221216161317,2022-12-16,city scrape,51 18 Caledonian Crescent,This beautiful third floor apartment is set in...,NaN,https://a0.muscache.com/pictures/prohost-api/H...,3810279,...,4.81,4.71,4.70,NaN,t,2,2,0,0,2.27
4,744710,https://www.airbnb.com/rooms/744710,20221216161317,2022-12-16,city scrape,Refurbished Flat in a Georgian Era Building in...,A stunning apartment in the heart of Edinburgh...,The apartment is in a Central Edinburgh neighb...,https://a0.muscache.com/pictures/monet/Select-...,3737047,...,4.96,4.96,4.86,NaN,t,1,1,0,0,1.73


In [188]:
#Delete all meaningless columns, like links, ids, etc.
listings_df = dirty_listings_df.drop(["id", "listing_url", "scrape_id", "last_scraped", "source" , "picture_url", "host_id", "host_url", "host_thumbnail_url", "host_picture_url", "latitude", "longitude", "calendar_last_scraped"], axis=1)

#Neighbourhood is the same everywhere, so we can delete it
listings_df = listings_df.drop(["neighbourhood"], axis=1)

#Description, neighbourhood overview and host_about is a text, and we assume that name of the host is not relevant, so we can delete it.
listings_df = listings_df.drop(["description", "neighborhood_overview", "host_name", "host_about"], axis=1)

#The name of the property is not relevant, so we can delete it. We still have indexes to identify the property.
listings_df = listings_df.drop(["name"], axis=1)


listings_df.head()

,host_since,host_location,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_neighbourhood,host_listings_count,host_total_listings_count,host_verifications,...,review_scores_communication,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,2009-12-06,"Edinburgh, United Kingdom",within a few hours,100%,96%,t,NaN,3,6,"['email', 'phone']",...,4.98,4.98,4.91,NaN,f,1,1,0,0,3.10
1,2012-07-29,"Edinburgh, United Kingdom",within a day,50%,84%,f,Old Town,9,16,"['email', 'phone']",...,4.72,4.94,4.09,NaN,t,8,6,2,0,0.55
2,2012-10-06,"Edinburgh, United Kingdom",within an hour,100%,95%,t,NaN,1,2,"['email', 'phone']",...,4.97,4.73,4.79,NaN,f,1,0,1,0,2.86
3,2012-10-09,"London, United Kingdom",within an hour,100%,99%,f,Haymarket,6,9,"['email', 'phone']",...,4.81,4.71,4.70,NaN,t,2,2,0,0,2.27
4,2012-10-02,"Edinburgh, United Kingdom",within an hour,100%,100%,t,New Town,1,1,"['email', 'phone']",...,4.96,4.96,4.86,NaN,t,1,1,0,0,1.73


In [189]:
#Delete all columns with more than 20% of missing values
listings_df = listings_df.dropna(thresh=len(listings_df) * 0.8, axis=1)
#Show the rest of missing values
listings_df.isna().sum()

host_since                                         0
host_location                                   1322
host_acceptance_rate                             726
host_is_superhost                                  2
host_listings_count                                0
host_total_listings_count                          0
host_verifications                                 0
host_has_profile_pic                               0
host_identity_verified                             0
neighbourhood_cleansed                             0
property_type                                      0
room_type                                          0
accommodates                                       0
bathrooms_text                                    13
bedrooms                                         138
beds                                             105
amenities                                          0
price                                              0
minimum_nights                                

In [190]:
#Delete all rows where reviews are missing, because that is the variable we want to predict
listings_df = listings_df.dropna(subset=["review_scores_rating", "review_scores_accuracy", "review_scores_cleanliness", "review_scores_checkin", "review_scores_communication", "review_scores_location", "review_scores_value"])
listings_df.isna().sum()

host_since                                         0
host_location                                   1087
host_acceptance_rate                             584
host_is_superhost                                  1
host_listings_count                                0
host_total_listings_count                          0
host_verifications                                 0
host_has_profile_pic                               0
host_identity_verified                             0
neighbourhood_cleansed                             0
property_type                                      0
room_type                                          0
accommodates                                       0
bathrooms_text                                     8
bedrooms                                         105
beds                                              60
amenities                                          0
price                                              0
minimum_nights                                

In [191]:
#Convert all dates to datetime
listings_df["host_since"] = pd.to_datetime(listings_df["host_since"])
listings_df["first_review"] = pd.to_datetime(listings_df["first_review"])
listings_df["last_review"] = pd.to_datetime(listings_df["last_review"])
listings_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6711 entries, 0 to 7388
Data columns (total 49 columns):
 #   Column                                        Non-Null Count  Dtype         
---  ------                                        --------------  -----         
 0   host_since                                    6711 non-null   datetime64[ns]
 1   host_location                                 5624 non-null   object        
 2   host_acceptance_rate                          6127 non-null   object        
 3   host_is_superhost                             6710 non-null   object        
 4   host_listings_count                           6711 non-null   int64         
 5   host_total_listings_count                     6711 non-null   int64         
 6   host_verifications                            6711 non-null   object        
 7   host_has_profile_pic                          6711 non-null   object        
 8   host_identity_verified                        6711 non-null   object

In [192]:
#Replace all missing values in the columns of type date with the average of the dates, and last review as well
listings_df["host_since"] = listings_df["host_since"].fillna(listings_df["host_since"].mean())
listings_df["first_review"] = listings_df["first_review"].fillna(listings_df["first_review"].mean())
listings_df["last_review"] = listings_df["last_review"].fillna(listings_df["last_review"].mean())

#Replace all the rest of missing values with the most frequent value in the column
listings_df = listings_df.fillna(listings_df.mode().iloc[0])

listings_df.isna().sum()

host_since                                      0
host_location                                   0
host_acceptance_rate                            0
host_is_superhost                               0
host_listings_count                             0
host_total_listings_count                       0
host_verifications                              0
host_has_profile_pic                            0
host_identity_verified                          0
neighbourhood_cleansed                          0
property_type                                   0
room_type                                       0
accommodates                                    0
bathrooms_text                                  0
bedrooms                                        0
beds                                            0
amenities                                       0
price                                           0
minimum_nights                                  0
maximum_nights                                  0


In [193]:
#Convert t and f to True and False
listings_df = listings_df.replace({"t": True, "f": False})

#Convert all percentages to float
listings_df["host_acceptance_rate"] = listings_df["host_acceptance_rate"].str.replace('%', '').astype('float') / 100.0
#Convert all prices to float
listings_df["price"] = listings_df["price"].str.replace('$', '').str.replace(',', '').astype('float')
#Convert number of beds and bedrooms to int
listings_df["beds"] = listings_df["beds"].astype('int')
listings_df["bedrooms"] = listings_df["bedrooms"].astype('int')

#Convert string representation of a list to string of comma-separated values
listings_df["host_verifications"] = listings_df["host_verifications"].str.replace('[', '').str.replace(']', '').str.replace("'", '')


listings_df.head()


/var/folders/rl/vtn0nzqd2mv0t2hrn6m_74540000gn/T/ipykernel_52105/3647658093.py:7: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  listings_df["price"] = listings_df["price"].str.replace('$', '').str.replace(',', '').astype('float')
/var/folders/rl/vtn0nzqd2mv0t2hrn6m_74540000gn/T/ipykernel_52105/3647658093.py:13: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  listings_df["host_verifications"] = listings_df["host_verifications"].str.replace('[', '').str.replace(']', '').str.replace("'", '')


,host_since,host_location,host_acceptance_rate,host_is_superhost,host_listings_count,host_total_listings_count,host_verifications,host_has_profile_pic,host_identity_verified,neighbourhood_cleansed,...,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,2009-12-06,"Edinburgh, United Kingdom",0.96,True,3,6,"email, phone",True,True,"Old Town, Princes Street and Leith Street",...,4.97,4.98,4.98,4.91,False,1,1,0,0,3.10
1,2012-07-29,"Edinburgh, United Kingdom",0.84,False,9,16,"email, phone",True,True,"Old Town, Princes Street and Leith Street",...,4.69,4.72,4.94,4.09,True,8,6,2,0,0.55
2,2012-10-06,"Edinburgh, United Kingdom",0.95,True,1,2,"email, phone",True,True,Polwarth,...,4.95,4.97,4.73,4.79,False,1,0,1,0,2.86
3,2012-10-09,"London, United Kingdom",0.99,False,6,9,"email, phone",True,True,Dalry and Fountainbridge,...,4.84,4.81,4.71,4.70,True,2,2,0,0,2.27
4,2012-10-02,"Edinburgh, United Kingdom",1.00,True,1,1,"email, phone",True,True,Deans Village,...,4.93,4.96,4.96,4.86,True,1,1,0,0,1.73


In [194]:
listings_df["bathrooms_text"].value_counts()

1 bath              3307
1 shared bath        997
2 baths              938
1.5 baths            486
1 private bath       452
1.5 shared baths     175
3 baths              107
2.5 baths             83
2 shared baths        51
3.5 baths             24
4 baths               21
0 baths               15
2.5 shared baths      13
5 baths                7
Shared half-bath       7
Half-bath              7
0 shared baths         6
5.5 baths              4
3 shared baths         2
6 baths                2
4.5 baths              2
6.5 baths              1
3.5 shared baths       1
7 baths                1
9 baths                1
9 shared baths         1
Name: bathrooms_text, dtype: int64

In [195]:

#Create a column indicating whether a bathroom is shared or private
listings_df.insert(14, "shared_bathroom",listings_df["bathrooms_text"].str.contains("shared"))

#Convert bathrooms_text to float
pattern = re.compile(r'\d+')
#If no number is found, assume 1 bathroom
listings_df["bathrooms_text"] = listings_df["bathrooms_text"].apply(lambda x: int(pattern.search(x).group()) if pattern.search(x) else 1)
listings_df.rename(columns={"bathrooms_text": "bathrooms"}, inplace=True)




listings_df.head()

,host_since,host_location,host_acceptance_rate,host_is_superhost,host_listings_count,host_total_listings_count,host_verifications,host_has_profile_pic,host_identity_verified,neighbourhood_cleansed,...,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,2009-12-06,"Edinburgh, United Kingdom",0.96,True,3,6,"email, phone",True,True,"Old Town, Princes Street and Leith Street",...,4.97,4.98,4.98,4.91,False,1,1,0,0,3.10
1,2012-07-29,"Edinburgh, United Kingdom",0.84,False,9,16,"email, phone",True,True,"Old Town, Princes Street and Leith Street",...,4.69,4.72,4.94,4.09,True,8,6,2,0,0.55
2,2012-10-06,"Edinburgh, United Kingdom",0.95,True,1,2,"email, phone",True,True,Polwarth,...,4.95,4.97,4.73,4.79,False,1,0,1,0,2.86
3,2012-10-09,"London, United Kingdom",0.99,False,6,9,"email, phone",True,True,Dalry and Fountainbridge,...,4.84,4.81,4.71,4.70,True,2,2,0,0,2.27
4,2012-10-02,"Edinburgh, United Kingdom",1.00,True,1,1,"email, phone",True,True,Deans Village,...,4.93,4.96,4.96,4.86,True,1,1,0,0,1.73


In [196]:
#Temporary
#listings_df["amenities"] = listings_df["amenities"].str.replace('[', '').str.replace(']', '').str.replace('"', '')
#amenities_expanded = listings_df['amenities'].str.get_dummies(sep=', ')
listings_df = listings_df.drop(["amenities"], axis=1)


In [197]:
host_verifications_expanded = listings_df['host_verifications'].str.get_dummies(sep=', ')
host_locations_expanded = listings_df['host_location'].str.get_dummies()
neighborhood_expanded = listings_df['neighbourhood_cleansed'].str.get_dummies()
property_type_expanded = listings_df['property_type'].str.get_dummies()
room_type_expanded = listings_df['room_type'].str.get_dummies()


# join the dummy variable columns with the original dataframe
listings_df = pd.concat([listings_df, host_verifications_expanded, host_locations_expanded, neighborhood_expanded, property_type_expanded, room_type_expanded], axis=1)
listings_df.head()

,host_since,host_location,host_acceptance_rate,host_is_superhost,host_listings_count,host_total_listings_count,host_verifications,host_has_profile_pic,host_identity_verified,neighbourhood_cleansed,...,Room in serviced apartment,Shared room in condo,Shared room in hostel,Shared room in vacation home,Tiny home,Windmill,Entire home/apt,Hotel room,Private room,Shared room
0,2009-12-06,"Edinburgh, United Kingdom",0.96,True,3,6,"email, phone",True,True,"Old Town, Princes Street and Leith Street",...,0,0,0,0,0,0,1,0,0,0
1,2012-07-29,"Edinburgh, United Kingdom",0.84,False,9,16,"email, phone",True,True,"Old Town, Princes Street and Leith Street",...,0,0,0,0,0,0,1,0,0,0
2,2012-10-06,"Edinburgh, United Kingdom",0.95,True,1,2,"email, phone",True,True,Polwarth,...,0,0,0,0,0,0,0,0,1,0
3,2012-10-09,"London, United Kingdom",0.99,False,6,9,"email, phone",True,True,Dalry and Fountainbridge,...,0,0,0,0,0,0,1,0,0,0
4,2012-10-02,"Edinburgh, United Kingdom",1.00,True,1,1,"email, phone",True,True,Deans Village,...,0,0,0,0,0,0,1,0,0,0
